#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [36]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [4]:
products = graphlab.SFrame('amazon_baby.gl/')

In [37]:
mec = graphlab.SFrame('data/sppmecdata.csv')

Finished parsing file /home/drew/Dev/spp-da-price-forecast/mlclass/data/sppmecdata.csv

Parsing completed. Parsed 100 lines in 0.096072 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/drew/Dev/spp-da-price-forecast/mlclass/data/sppmecdata.csv

Parsing completed. Parsed 3648 lines in 0.043403 secs.

#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [5]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [9]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [10]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, '6': 1,'stink': 1, 'because' ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."


In [38]:
graphlab.canvas.set_target('ipynb')

In [11]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [12]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']


In [13]:
len(giraffe_reviews)

785

In [14]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [15]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [16]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [39]:
#positive sentiment = 4* or 5* reviews
#products['sentiment'] = products['rating'] >=4
mec['buysig'] = mec['diffmec'] < 0

In [20]:
products.head()
mec.tail()

interval,hour,windforecast,loadforecast,natgasprice,diffmec,buysig
2017-02-28 14:00:00,14.0,9425.5,27292.0,2.54,-2.2532,1
2017-02-28 15:00:00,15.0,9471.51,27128.0,2.54,-2.5527,1
2017-02-28 16:00:00,16.0,9647.9,27016.0,2.54,-5.6112,1
2017-02-28 17:00:00,17.0,9936.06,27060.0,2.54,-75.61,1
2017-02-28 18:00:00,18.0,10276.78,27300.0,2.54,-9.7366,1
2017-02-28 19:00:00,19.0,10789.89,28246.0,2.54,-4.636,1
2017-02-28 20:00:00,20.0,11119.76,28423.0,2.54,-20.174,1
2017-02-28 21:00:00,21.0,11434.75,28229.0,2.54,-3.5214,1
2017-02-28 22:00:00,22.0,11755.12,27325.0,2.54,-1.1856,1
2017-02-28 23:00:00,23.0,12078.26,25891.0,2.54,3.7234,0


##Let's train the sentiment classifier

In [40]:
#train_data,test_data = products.random_split(.8, seed=0)
trdata,tedata = mec.random_split(.8, seed=0)

In [41]:
#sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     #target='sentiment',
                                                     #features=['word_count'],
                                                     #validation_set=test_data)
sppfeatures = ['loadforecast','windforecast','natgasprice','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
spp_model = graphlab.logistic_classifier.create(trdata,
                                                     target='buysig',
                                                     features=sppfeatures,
                                                     validation_set=tedata)

Logistic regression:

--------------------------------------------------------

Number of examples          : 2928

Number of classes           : 2

Number of feature columns   : 27

...

| 2         | 3        | 0.102761     | 0.761954          | 0.755556            |

| 3         | 4        | 0.149100     | 0.762295          | 0.755556            |

| 4         | 5        | 0.190846     | 0.762295          | 0.755556            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

#Evaluate the sentiment model

In [42]:
#sentiment_model.evaluate(test_data, metric='roc_curve')
spp_model.evaluate(tedata, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-----+-----+
 | threshold | fpr | tpr |  p  |  n  |
 +-----------+-----+-----+-----+-----+
 |    0.0    | 1.0 | 1.0 | 176 | 544 |
 |   1e-05   | 1.0 | 1.0 | 176 | 544 |
 |   2e-05   | 1.0 | 1.0 | 176 | 544 |
 |   3e-05   | 1.0 | 1.0 | 176 | 544 |
 |   4e-05   | 1.0 | 1.0 | 176 | 544 |
 |   5e-05   | 1.0 | 1.0 | 176 | 544 |
 |   6e-05   | 1.0 | 1.0 | 176 | 544 |
 |   7e-05   | 1.0 | 1.0 | 176 | 544 |
 |   8e-05   | 1.0 | 1.0 | 176 | 544 |
 |   9e-05   | 1.0 | 1.0 | 176 | 544 |
 +-----------+-----+-----+-----+-----+
 [100001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [43]:
#sentiment_model.show(view='Evaluation')
spp_model.show(view='Evaluation')

#Applying the learned model to understand sentiment for Giraffe

In [44]:
#giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')
mec['predicted_buysig'] = spp_model.predict(mec, output_type='probability')

In [46]:
mecmec.sort('predicted_buysig', ascending=False)
mec.head()

interval,hour,windforecast,loadforecast,natgasprice,diffmec,0,1,2,3,4,5
2016-08-01 00:00:00,0.0,9419.15,34925.0,2.97,5.5041,1.0,0.0,0.0,0.0,0.0,0.0
2016-08-01 01:00:00,1.0,9526.57,33084.0,2.97,6.5692,0.0,1.0,0.0,0.0,0.0,0.0
2016-08-01 02:00:00,2.0,9508.95,31562.0,2.97,3.9592,0.0,0.0,1.0,0.0,0.0,0.0
2016-08-01 03:00:00,3.0,9376.61,30472.0,2.97,2.8883,0.0,0.0,0.0,1.0,0.0,0.0
2016-08-01 04:00:00,4.0,9228.14,29909.0,2.97,3.292,0.0,0.0,0.0,0.0,1.0,0.0
2016-08-01 05:00:00,5.0,9039.04,29803.0,2.97,3.5476,0.0,0.0,0.0,0.0,0.0,1.0
2016-08-01 06:00:00,6.0,8802.81,30575.0,2.97,5.8657,0.0,0.0,0.0,0.0,0.0,0.0
2016-08-01 07:00:00,7.0,8544.92,31921.0,2.97,3.6058,0.0,0.0,0.0,0.0,0.0,0.0
2016-08-01 08:00:00,8.0,8215.24,33705.0,2.97,3.3288,0.0,0.0,0.0,0.0,0.0,0.0
2016-08-01 09:00:00,9.0,7676.61,35848.0,2.97,1.4762,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1, 'all': 1,'because': 1, 'it': 1, ...",0.999513023521
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1, 'right': 1,'help': 1, 'just': 1, ...",0.999320678306
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2, 'all': 1,'would': 1, 'latex.': 1, ...",0.013558811687
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2, 'one!': 1,'all': 1, 'love': 1, ...",0.995769474148
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3, 'all': 1,'months': 1, 'old': 1, ...",0.662374415673
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6, 'seven': 1,'already': 1, 'love': 1, ...",0.999997148186
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4, 'drooling,':1, 'love': 1, ...",0.989190989536
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3, 'chew': 1,'be': 1, 'is': 1, ...",0.999563518413
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2, 'seemed': 1,'because': 1, 'about.': ...",0.970160542725
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2, 'already': 1,'some': 1, 'it': 3, ...",0.195367644588


##Sort the reviews based on the predicted sentiment and explore

In [22]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [24]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1, 'all': 1,""violet's"": 2, 'food' ...",1.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'peace': 1, 'month': 1,'bright': 1, 'softer' ...",0.999999999703
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2, 'pops': 1,'existence.': 1, ...",0.999999999392
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2, ""don't"": 1,'(literally).so': 1, ...",0.99999999919
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1, 'all': 1,'reviews.': 2, 'just' ...",0.999999998657
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2, 'both': 1,'month': 1, 'ears,': 1, ...",0.999999997108
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5, 'the': 1,'all': 1, 'that': 2, ...",0.999999995589
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1, 'shape': 2,'mutt': 1, '""dog': 1, ...",0.999999995573
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4, 'chew': 1,'all': 1, 'perfect;': 1, ...",0.999999989527
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1, 'all': 1,'out,': 1, 'over': 1, ...",0.999999985069


##Most positive reviews for the giraffe

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [28]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."